In [4]:
# Lab 12 RNN
import torch
import torch.nn as nn
from torch.autograd import Variable

torch.manual_seed(777)  # reproducibility


idx2char = ['h', 'i', 'e', 'l', 'o']

# Teach hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

y_data = [1, 0, 2, 3, 3, 4]    # ihello

# As we have one batch of samples, we will change them to variables only once
inputs = Variable(torch.Tensor(x_one_hot))
labels = Variable(torch.LongTensor(y_data))
print(inputs)
print(labels)

tensor([[[1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 0.]]])
tensor([1, 0, 2, 3, 3, 4])


In [5]:

num_classes = 5
input_size = 5  # one-hot size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6
num_layers = 1  # one-layer rnn


class RNN(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(RNN, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length

        self.rnn = nn.RNN(input_size=5, hidden_size=5, batch_first=True)

    def forward(self, x):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size) for batch_first=True
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))

        # Reshape input
        x.view(x.size(0), self.sequence_length, self.input_size)

        # Propagate input through RNN
        # Input: (batch, seq_len, input_size)
        # h_0: (num_layers * num_directions, batch, hidden_size)

        out, _ = self.rnn(x, h_0)
        return out.view(-1, num_classes)


# Instantiate RNN model
rnn = RNN(num_classes, input_size, hidden_size, num_layers)
print(rnn)



RNN(
  (rnn): RNN(5, 5, batch_first=True)
)


In [8]:
# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.1)

# Train the model
for epoch in range(100):
    outputs = rnn(inputs)
    optimizer.zero_grad()
    print(outputs)
    print(labels)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    result_str = [idx2char[c] for c in idx.squeeze()]
    print("epoch: %d, loss: %1.3f" % (epoch + 1, loss.item()))
    print("Predicted string: ", ''.join(result_str))

print("Learning finished!")

tensor([[-1.0000,  0.9999, -1.0000, -1.0000, -0.9998],
        [ 1.0000, -1.0000, -1.0000, -0.9997, -1.0000],
        [-1.0000, -0.9998,  1.0000, -0.9998, -1.0000],
        [-1.0000, -1.0000, -1.0000,  1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  0.9918, -0.3271],
        [-1.0000, -1.0000, -1.0000, -0.9558,  0.9653]], grad_fn=<ViewBackward>)
tensor([1, 0, 2, 3, 3, 4])
epoch: 1, loss: 0.449
Predicted string:  ihello
tensor([[-1.0000,  1.0000, -1.0000, -1.0000, -0.9998],
        [ 1.0000, -1.0000, -1.0000, -0.9998, -1.0000],
        [-1.0000, -0.9999,  1.0000, -0.9998, -1.0000],
        [-1.0000, -1.0000, -1.0000,  1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  0.9731,  0.2546],
        [-1.0000, -0.9999, -1.0000, -1.0000,  0.9999]], grad_fn=<ViewBackward>)
tensor([1, 0, 2, 3, 3, 4])
epoch: 2, loss: 0.468
Predicted string:  ihello
tensor([[-1.0000,  1.0000, -1.0000, -1.0000, -0.9999],
        [ 1.0000, -1.0000, -1.0000, -0.9998, -1.0000],
        [-1.0000, -0.9999,  1.

epoch: 77, loss: 0.448
Predicted string:  ihello
tensor([[-1.0000,  1.0000, -1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000,  1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  0.9946, -0.3550],
        [-1.0000, -1.0000, -1.0000, -0.9817,  0.9721]], grad_fn=<ViewBackward>)
tensor([1, 0, 2, 3, 3, 4])
epoch: 78, loss: 0.448
Predicted string:  ihello
tensor([[-1.0000,  1.0000, -1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000,  1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  0.9948, -0.3627],
        [-1.0000, -1.0000, -1.0000, -0.9787,  0.9701]], grad_fn=<ViewBackward>)
tensor([1, 0, 2, 3, 3, 4])
epoch: 79, loss: 0.448
Predicted string:  ihello
tensor([[-1.0000,  1.0000, -1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000